# Setting Env (Selenium, Chrome)

In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 4.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://devel

# Google Drive Mount


In [0]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Chrome Setting

In [0]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  
from selenium.common.exceptions import TimeoutException

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

import pandas as pd
import traceback

import time

import pdb
import re
import numpy as np

In [0]:
def search_with_exception(selector, time = 1):
  try:
    ele = WebDriverWait(driver, time).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    print(ele.text)
    return ele
  except TimeoutException:
    traceback.print_exc()
    
def execute_script(script, time = 5):
  driver.set_script_timeout(time)
  driver.execute_script(script)
  #print("execute script : {script}".format(script=script))
  
def get_button_script(source, location):
  query1 = 'javascript.*{}'.format(location)
  query2 = ':.*?\)'
 
  source = re.findall(query1,source)[0]
  script = re.findall(query2,source)[0][1:].strip()
  #print(script)
  return script

def recursive(location):
  #print(location)
  for curr in location:
    html = driver.page_source.split('\n')
   
    for line in html:
      line = line.strip()
      
      if re.search('{}'.format(curr),line):
        execute_script(get_button_script(line, curr))
        time.sleep(1)

# 시군구 얻어오기

In [0]:
url = "https://www.water.or.kr"
path = '/content/gdrive/My Drive/'
name = 'water_quality.csv'

driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)  

data = {}
df = pd.DataFrame({'location1':[], 'location2':[], 'location3':[]})
i = 0

main_handle = driver.current_window_handle

execute_script("fnOpenCenterWindow1('/kwip/common/popupLocation.do','430','330','','')",5)

for handle in driver.window_handles:
  if handle != main_handle:
    sub_handle = handle

#handle 전환 
driver.implicitly_wait(2)
driver.switch_to.window(sub_handle)
time.sleep(1)

#wait server side
시도 = driver.find_elements_by_css_selector('#sido_nm > li')

for index, ele in enumerate(시도):
  if index != 0:
    ele.click()
    time.sleep(1)
    
    군구 = driver.find_elements_by_css_selector('#sigun_nm > li')
   
    for index, ele in enumerate(군구):
      if index != 0:
        ele.click()
        time.sleep(1)
      
        #읍면동 = driver.find_elements_by_css_selector('#eupmyeondong > li')
        읍면동 = driver.find_element_by_css_selector('#eupmyeondong')
        읍면동 = 읍면동.text.split('\n')
        
        for index, ele in enumerate(읍면동):
          if index != 0:
            current = driver.find_elements_by_css_selector('.on')

            df.loc[i, 'location1'] = current[0].text
            df.loc[i, 'location2'] = current[1].text
            df.loc[i, 'location3'] = ele
            i += 1

            print("{}/{}/{}/{}".format(i, current[0].text, current[1].text, ele))


# 수질 정보 얻어오기

In [0]:
url = "https://www.water.or.kr"
path = '/content/gdrive/My Drive/'
name = 'water_quality.csv'

driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)  

df = pd.read_csv(path + name)
df = df[df.location2 != "강남구"]
df = df[df.location2 != "강동구"]
df['blur'] = np.nan
df['chlorine'] = np.nan
df['ph'] = np.nan



for index, row in df.iterrows():
  
  시도 = row['location1']
  시군 = row['location2']
  읍면동 = row['location3']
  print('{} {} {}'.format(시도,시군, 읍면동))
    
  #크롬 새로고침
  print('1')
  driver.refresh()
  driver.implicitly_wait(5)
  
  #handle 전환 
  main_handle = driver.current_window_handle
  execute_script("fnOpenCenterWindow1('/kwip/common/popupLocation.do','430','330','','')",10)

  for handle in driver.window_handles:
    if handle != main_handle:
      sub_handle = handle

  driver.implicitly_wait(2)
  driver.switch_to.window(sub_handle)
  time.sleep(1)


  #지역별 접근
  print('2')
  
  recursive([시도, 시군, 읍면동])
  execute_script("fn_popup_submit()")
  driver.switch_to.window(main_handle)
  time.sleep(1)
  
  #데이터 크롤링
  print('3')
  
  data = ['#blur', '#chlorine', '#ph']
  print(4)
  pdb.set_trace()
  driver.refresh()
  driver.implicitly_wait(5)
  print(5)
  ele = driver.find_element_by_css_selector('.turbidity')
  print(6)
  
  df.loc[index, 'blur'] = ele.find_element_by_css_selector(data[0]).text
  df.loc[index, 'chlorine'] = ele.find_element_by_css_selector(data[1]).text
  df.loc[index, 'ph'] = ele.find_element_by_css_selector(data[2]).text
    
  print('탁도 : {}, 잔류염소 : {}, ph : {}'.format(df.loc[index, 'blur'], df.loc[index, 'chlorine'], df.loc[index, 'ph']))
  print('')
    
  


In [0]:
df.to_csv(path+name)